In [1]:
# Indicates where the projection database is prior to importing fiona
import os
os.environ['PROJ_LIB'] = r'C:\Users\osori050\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\Library\share\proj'

In [2]:
import arcpy
from arcpy.sa import *
import requests
import zipfile
import io
import json
from shapely import geometry
from fiona.crs import from_epsg
import fiona

In [3]:
arcpy.env.workspace = r'E:\ArcGIS_1\Lab2\Dory'
workspace = arcpy.env.workspace

In [4]:
# Retrieves the location of the North Picnic Area park from Google Places 
north_picnic_link = r'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=North%20Picnic%20area%20St%20Charles%20Minnesota&inputtype=textquery&fields=formatted_address%2Cname%2Crating%2Copening_hours%2Cgeometry&key=YOUR_API_KEY'
north_picnic = requests.get(north_picnic_link)
north_picnic_dic = json.loads(north_picnic.text)
coords = north_picnic_dic['candidates'][0]['geometry']['location']
north_picnic_location = [float(coords['lng']), float(coords['lat'])]

# Dory's house
house = [-92.148796, 44.127985,]

In [5]:
# Creates shapefiles with the coordinates of the start and end points

dory_schema =  {'geometry': 'Point', 'properties': {'location': 'str'}}

with fiona.open(r"points.shp", 'w', crs = from_epsg(4326), driver = 'ESRI Shapefile', schema = dory_schema) as output:
    points = [geometry.Point(house[0], house[1]), geometry.Point(north_picnic_location[0], north_picnic_location[1])]
    location = ['Start point', 'End point']
    for i in range(2):
        prop = {'location': location[i]}
        output.write({'geometry': geometry.mapping(points[i]), 'properties': prop})

In [6]:
# Projects to NAD83 UTM Zone 15N, creates a bounding box around the start and end points, 
# and creates an 8-km buffer to consider land beyond the bounding box in the analysis 
arcpy.management.Project(r"points.shp", r"points_Project.shp", 'PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', "WGS_1984_(ITRF00)_To_NAD_1983", 'GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', "NO_PRESERVE_SHAPE", None, "NO_VERTICAL")
arcpy.management.MinimumBoundingGeometry(r"points_Project.shp", r"polygon.shp", "ENVELOPE", "ALL", None, "NO_MBG_FIELDS")
arcpy.analysis.Buffer(r"polygon.shp", r"AOI.shp", "8 Kilometers", "FULL", "ROUND", "ALL", None, "PLANAR")

<Result 'E:\\ArcGIS_1\\Lab2\\Dory\\AOI.shp'>

## Slope
https://gisdata.mn.gov/dataset/elev-30m-digital-elevation-model

In [7]:
# Retrieves DEM from MGC
dem_output = requests.post(r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/elev_30m_digital_elevation_model/fgdb_elev_30m_digital_elevation_model.zip')
zipfile.ZipFile(io.BytesIO(dem_output.content)).extractall(workspace)

In [8]:
# Clips DEM to AOI, creates slope raster, and reclassifies using geometric interval methods as increments in higher slopes
# are penalized more severely
arcpy.management.Clip(r"elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m", "560098.327934821 4870356.13572893 584510.985228164 4894439.34794601", r"dem.tif", r"AOI.shp", "32767", "NONE", "NO_MAINTAIN_EXTENT")
out_raster = arcpy.sa.Slope(r"dem.tif", "DEGREE", 1, "PLANAR", "METER"); out_raster.save(r"slope.tif")
out_raster = arcpy.sa.Reclassify(r"slope.tif", "VALUE", "0 3.078800 1;3.078800 10.647242 2;10.647242 29.252321 3;29.252321 74.988144 4", "DATA"); out_raster.save(r"Reclass_slope.tif")

## Farm fields
https://gisdata.mn.gov/dataset/agri-cropland-data-layer-2021

In [9]:
# Retrieves farm field information from the Agricultural cropland data layer 2021 from MGC
farm_fields = requests.post(r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_mda/agri_cropland_data_layer_2021/fgdb_agri_cropland_data_layer_2021.zip')
zipfile.ZipFile(io.BytesIO(farm_fields.content)).extractall(workspace)

In [10]:
# Clips to AOI and reclassifies
arcpy.management.Clip(r"agri_cropland_data_layer_2021.gdb\agri_cropland_data_layer_2021", "560098.327934821 4870356.13572893 584510.985228164 4894439.34794601", r"farm_fields.tif", r"AOI.shp", "32767", "NONE", "NO_MAINTAIN_EXTENT")
out_raster = arcpy.sa.Reclassify(r"farm_fields.tif", "CLASS_NAME", "Corn 4;Sorghum 4;Soybeans 4;'Sweet Corn' 4;Barley 4;'Spring Wheat' 4;'Winter Wheat' 4;Rye 4;Oats 4;Alfalfa 4;'Other Hay/Non Alfalfa' 4;Sugarbeets 4;'Dry Beans' 4;Potatoes 4;Peas 4;Clover/Wildflowers 3;'Sod/Grass Seed' 4;Switchgrass 3;'Fallow/Idle Cropland' 4;Apples 4;'Open Water' 4;'Developed/Open Space' 1;'Developed/Low Intensity' 1;'Developed/Med Intensity' 1;'Developed/High Intensity' 1;Barren 1;'Deciduous Forest' 2;'Evergreen Forest' 2;'Mixed Forest' 2;Shrubland 2;Grassland/Pasture 3;'Woody Wetlands' 4;'Herbaceous Wetlands' 4", "DATA"); out_raster.save(r"Reclass_farm_fields.tif")

## Water
https://gisdata.mn.gov/dataset/water-mn-public-waters

In [11]:
# Retrieves watercourse layers from MGC
watercourses = requests.post(r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dnr/water_mn_public_waters/shp_water_mn_public_waters.zip')
zipfile.ZipFile(io.BytesIO(watercourses.content)).extractall(workspace)

In [12]:
# Clips to AOI
arcpy.analysis.Clip(r"public_waters_watercourses_delineations.shp", r"AOI.shp", r"watercourse_Clip.shp", None)
# Polyline to raster
arcpy.conversion.PolylineToRaster(r"watercourse_Clip.shp", "FID", r"watercourse_Raster.tif", "MAXIMUM_LENGTH", "NONE", 30, "BUILD")
# Reclassifies river
out_raster = arcpy.sa.Reclassify(r"watercourse_Raster.tif", "Value", "0 115 4;NODATA 0", "DATA"); out_raster.save(r"Reclass_water.tif")

## Bridges
https://gisdata.mn.gov/dataset/trans-bridges

In [13]:
# Retrieves bridges layers from MGC
bridges = requests.post(r'https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_dot/trans_bridges/shp_trans_bridges.zip')
zipfile.ZipFile(io.BytesIO(bridges.content)).extractall(workspace)

In [14]:
# Buffer to only include bridges within 30 meters of the watercourses 
arcpy.analysis.Buffer(r"watercourse_Clip.shp", r"watercourse_Buffer.shp", "30 Meters", "FULL", "ROUND", "ALL", None, "PLANAR")
arcpy.analysis.Clip(r"Bridge_locations_in_Minnesota.shp", r"watercourse_Buffer.shp", r"bridges_clip.shp", None)
# As the shapefile is a point vector layer, a Snap is required to place bridges precisely on top of watercourses
arcpy.edit.Snap(r"bridges_clip.shp", "watercourse_Clip.shp EDGE '50 Meters'")
# Point to raster
arcpy.conversion.PointToRaster(r"bridges_clip.shp", "FID", r"bridges__Raster.tif", "MOST_FREQUENT", "NONE", 30, "BUILD")
# Reclassifies bridges with the same value as watercourses 
out_raster = arcpy.sa.Reclassify(r"bridges__Raster.tif", "Value", "0 109 4;NODATA 0", "DATA"); out_raster.save(r"Reclass_bridge.tif")

## Euclidean Distance

In [15]:
# Calculates the euclidean distance between the start and end points (a line), and then the euclidean distance from this line
# to the rest of the AOI
arcpy.management.PointsToLine("points_Project.shp", r"Euclidean_distance.shp", None, None, "NO_CLOSE")
with arcpy.EnvManager(extent='560095.332039 4870335.370038 584515.332039 4894455.370038 PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'):
    out_distance_raster = arcpy.sa.EucDistance("Euclidean_distance.shp", None, 30, None, "PLANAR", None, None); out_distance_raster.save(r"EucDist.tif")

# Reclassifies the euclidean distance to the line using the geometric interval (same as with the slope above) 
out_raster = arcpy.sa.Reclassify("EucDist.tif", "VALUE", "0 4044.337550 1;4044.337550 6547.644254 2;6547.644254 10591.981804 3;10591.981804 17126.005859 4", "DATA"); out_raster.save(r"Reclass_EucDist.tif")

## Cost surface
This block of code creates different cost surface rasters based on the weight factors inputted by the user and saves the datasets to disk. It stops when the user specifies they do not want to create a new cost surface by typing "no" in the input box.

In [20]:
# Keeps track of the cost surfaces created by the user
counter = 0

while True:
    question = input('New cost surface?')
    
    if question.lower() == 'yes':
        try:
            v = float(input('Enter weighting factor for distance'))
            w = float(input('Enter weighting factor for farm fields'))
            x = float(input('Enter weighting factor for water'))
            y = float(input('Enter weighting factor for bridge. It can be negative if you want to counterbalance the cost of crossing water bodies'))
            z = float(input('Enter weighting factor for slope'))
        except:
            raise Exception ('Please enter numeric input')
            
        # Changes the name of the output in each cycle not to overwrite the datasets
        counter += 1
        output_name = workspace + r"\cost_surface_" + str(counter) + ".tif"

        # Map algebra 
        algebra = v*Raster("Reclass_EucDist.tif") + w*Raster("Reclass_farm_fields.tif") + x*Raster("Reclass_water.tif") + y*Raster("Reclass_bridge.tif") + z*Raster("Reclass_slope.tif")
        algebra.save(output_name)
        
    elif question.lower() == 'no':
        break
    else:
        print('Please enter only "yes" or "no"')

New cost surface?yes
Enter weighting factor for distance0.5
Enter weighting factor for farm fields2
Enter weighting factor for water3
Enter weighting factor for bridge. It can be negative if you want to counterbalance the cost of crossing water bodies-2
Enter weighting factor for slope1
New cost surface?no
